In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 80)
from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

## Classifying investor purchases

Process to classify deeds as (a) residential properties, (b) likely purchased by investors.

## 1. Selecting deeds that fall within residential parcels

Using land use data from the city of Philadelphia (see original source [here](https://www.opendataphilly.org/dataset/land-use))

In [2]:
# read in land use file
landuse = gpd.read_file("../00_data/RAW/philadelphia/land_use/Land_Use_2019/Land_Use.shp")

In [3]:
# creating a more narrow set of residential parcels
# just low and medium residential uses
res = landuse.loc[(landuse['C_DIG2DESC']=='Residential Medium') | 
                    (landuse['C_DIG2DESC']=='Residential Low')]

# dropping pacels with na values for their geometry
res = res.dropna(subset=['geometry'])

# reprojecting residential low/med only category
res = res.to_crs('epsg:2272')

In [4]:
# reading in the real estate tax transfer data
rtt = pd.read_csv("../00_data/RAW/philadelphia/realestate_tax_transfers_2021/rtt_summary.csv")

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (26,34,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# selecting just deeds (not mortgages and other transfer taxes)
deeds = rtt.loc[rtt['document_type'] == "DEED"]
# adding a year column from the document date
deeds["sort_year"] = deeds['display_date'].astype(str).str[:4]

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
deeds

objectid  document_id document_type         display_date  \
319       7741116     50128982          DEED  2000-08-25 00:00:00   
320       7741117     50128987          DEED  2000-08-25 00:00:00   
465       7741398     51075355          DEED  2004-12-07 00:00:00   
470       7741283     50791375          DEED  2003-10-10 00:00:00   
474       7741287     50853599          DEED  2004-01-21 00:00:00   
...           ...          ...           ...                  ...   
4172524  11937539     53751061          DEED  2020-11-05 00:00:00   
4172541  11937977     53751528          DEED  2020-11-12 00:00:00   
4172542  11937978     53751529          DEED  2020-11-11 00:00:00   
4172549  11937985     53751536          DEED  2020-11-09 00:00:00   
4172552  11937988     53751539          DEED  2020-10-30 00:00:00   

             street_address  zip_code  ward  \
319                     NaN       NaN   NaN   
320                     NaN       NaN   NaN   
465                     NaN       NaN   NaN   
470                     NaN       NaN   NaN   
474                     NaN       NaN   NaN   
...                     ...       ...   ...   
4172524      1739 S 12TH ST   19148.0  39.0   
4172541  4709 FRANKFORD AVE   19124.0  23.0   
4172542      1305 N 51ST ST   19131.0  44.0   
4172549    158 GREENDALE RD   19154.0  66.0   
4172552    4435 MITCHELL ST   19128.0  21.0   

                                                  grantors  \
319                                       MISSING DOCUMENT   
320                                       MISSING DOCUMENT   
465      PHILADELPHIA AUTHORITY FOR INDUSTRIAL DEVELOPMENT   
470                                     DEL VALLE PENNYS M   
474                           COMMONWEALTH OF PENNSYLVANIA   
...                                                    ...   
4172524                        HUANG QI PING; LU RONG YUAN   
4172541                                   NGUYEN THANH KIM   
4172542                                       BS PROPS LLC   
4172549                                         BYRNE SEAN   
4172552                   SHAPIRO ERIC M; SHAPIRO NICOLE N   

                                 grantees  cash_consideration  \
319                      MISSING DOCUMENT                 1.0   
320                      MISSING DOCUMENT                 1.0   
465            PIDC FINANCING CORPORATION             33600.0   
470                       DEJESUS NAZARIO             50000.0   
474                     BEACH STREET CORP             71150.0   
...                                   ...                 ...   
4172524              510 PHILADELPHIA LLC            385000.0   
4172541                2906-8 N FIVE CORP            175000.0   
4172542  TCS ANIKA HOMES ACQUISITIONS III             62500.0   
4172549    GORHAM MALLORY; GORHAM PATRICK            305000.0   
4172552        DARABI ALI; TOFIGH MAHSHID            295000.0   

         other_consideration  total_consideration  assessed_value  \
319                      NaN                  1.0             NaN   
320                      NaN                  1.0             NaN   
465                      NaN              33600.0             NaN   
470                      NaN              50000.0             NaN   
474                      0.0              71150.0             NaN   
...                      ...                  ...             ...   
4172524                  0.0             385000.0        500900.0   
4172541                  0.0             175000.0        137200.0   
4172542                  0.0              62500.0         55300.0   
4172549                  0.0             305000.0             0.0   
4172552                  0.0             295000.0        241600.0   

         common_level_ratio  fair_market_value  state_tax_amount  \
319                     NaN           16464.00               0.0   
320                     NaN          164640.00               0.0   
465                     NaN               0.00          

In [7]:
# defining points from lat/lon
deeds_points = [Point(xy) for xy in zip(deeds.lat,deeds.lng)]
deedsgeo = GeoDataFrame(deeds, geometry=deeds_points)

In [8]:
# selecting only deeds with non zero state taxes
deedsgeo = deedsgeo.loc[deedsgeo["state_tax_amount"] > 0]

# and that also have an address field
deedsgeo= deedsgeo.loc[deedsgeo["address_low"]>0]

In [9]:
# setting the projection for the lat lon coordinates 
deedsgeo.crs = 'epsg:4326'

In [10]:
# reprojecting to: EPSG:2272: NAD83 / Pennsylvania South (ftUS)
deedsgeo = deedsgeo.to_crs('epsg:2272')

In [11]:
# spatial join between deeds and residential parcels (to find only those deeds within residential parcels)
deeds_res = gpd.sjoin(deedsgeo,res, how="inner",op="within")

## 2. Identify likely investor purchases
First tag all grantees with corporate entity names. Then find all investors who purchased more than 2 properties in any one year span.   

### 2A. Identifying Deeds with likely investors as the grantee based on grantee name
First tag all investor named entities   
  - LLC
  - LP
  - LIMITED PARTNERSHIP
  - CORP
  - CORPORATION
  - INC
  - COMPANY
  - LIMITED PARTNE
  - L P
  - L L C
  - PARTNERSHIP
  - LTD
  - TRUST 
    


In [12]:
# selecting just post-2000 deeds
deeds_res['sort_year'] = pd.to_numeric(deeds_res['sort_year'])
deeds_res = deeds_res.loc[deeds_res['sort_year']>1999]

# selecting investors by grantee name
inv = deeds_res['grantees'].str.contains(
    'LLC|LP|LIMITED PARTNERSHIP|CORP|CORPORATION|INC|COMPANY|LIMITED PARTNE|L P|L L C|PARTNERSHIP|LTD|TRUST'
    ,regex=True)

# converting investors series to dataframe
inv = inv.to_frame()

# merging with deeds
deeds_inv = deeds_res.merge(
    inv, left_index=True, right_index = True)
deeds_inv = deeds_inv.dropna(subset=['grantees_x'])

# creating a count column to allow grouping and count 
# for investor and owner by year 
deeds_inv['inv'] = deeds_inv['grantees_y'].apply(lambda x: 1 if x == True else 0)
deeds_inv['occ'] = deeds_inv['grantees_y'].apply(lambda x: 1 if x == False else 0)

# dropping duplicate object ids from the dataset
deeds_inv = deeds_inv.drop_duplicates(subset="objectid")

### 2B. Classifying investor purchases by multiple purchasers
Identifying grantees who purchased more than 2 properties in any one year period from the full set of non-investor classified deeds:

In [13]:
# finding multiple purchasers from the full set of non-investor classified deeds
# selecting from just the non llc llp deeds
occ = deeds_inv.loc[deeds_inv['occ']==1]

In [14]:
# getting all deeds from purchasers who are associated with more than one deed
multiple_purchasers = occ.groupby(
    "grantees_x").filter(lambda s: s['objectid'].count()>=2)


In [15]:
# converting date to datetime format
multiple_purchasers['display_date'] = multiple_purchasers[
    'display_date'].astype('datetime64[ns]')

In [16]:
# dropping unneeded columns
columns_multi = ['objectid','grantors','grantees_x',
                 'display_date','sort_year','street_address']
multiple_purchase_simple = multiple_purchasers[columns_multi]


In [17]:

multiple_purchase_simple[
    'elapsed'] = multiple_purchase_simple.sort_values(
    ['grantees_x','display_date']).groupby(
    'grantees_x')['display_date'].diff()

# converting timedelta column to an int type 
# to make it possible to filter below
multiple_purchase_simple[
    'elapsed_days'] = multiple_purchase_simple[
    'elapsed'].dt.days


/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
# filtering for deeds from grantees who purchased two properties within 1 year of each other
multiple_investors = multiple_purchase_simple.groupby('grantees_x').filter(
    lambda x: x['elapsed_days'].min() <= 365)
multiple_investors['multi_invest'] = 1

## 4. merging multiple purchase investors back with full set of deeds

Compiling a comprehensive classification of investor purchases 

In [19]:
deeds_multi = deeds_inv.merge(
    multiple_investors, left_on="objectid", right_on="objectid",how="left")

deeds_multi["investor_all"] = deeds_multi[
    "inv"] + deeds_multi["multi_invest"].fillna(0)
deeds_multi['owner_occup'] = deeds_multi[
    'investor_all'].apply(lambda x: 0 if x == 1 else 1)

In [20]:
deeds_multi.head()

objectid  document_id document_type       display_date_x  \
0   7744699     50942296          DEED  2004-04-02 00:00:00   
1   7744935     50092387          DEED  2000-05-25 00:00:00   
2   7744943     50942514          DEED  2004-05-28 00:00:00   
3   7744945     50942516          DEED  2004-05-28 00:00:00   
4   7744231     50941882          DEED  2004-05-12 00:00:00   

      street_address_x  zip_code  ward  \
0      3083 MEMPHIS ST   19134.0  25.0   
1       3631 AVALON ST   19114.0  66.0   
2       3430 ALDINE ST   19136.0  64.0   
3        5809 AKRON ST   19149.0  62.0   
4  1403 ROSE PETAL WAY   19111.0  56.0   

                                          grantors_x       grantees_x_x  \
0              MCCAFFERTY MARIE E; TUCKER MARGARET T    IRWIN LEIGHANNE   
1  TAYLOR COLERIDGE; TAYLOR ROBERT TR; TAYLOR RON...  FALLEN PAUL J III   
2                        ROWLAND JAMES; ROWLAND JOAN      RUTA JOSEPH M   
3                                         LAVIN BETH       DUFFIE JAMIE   
4                   CHAMPION DEVELOPMENT CORPORATION            NVR INC   

   cash_consideration  other_consideration  total_consideration  \
0             58900.0                  NaN              58900.0   
1             92000.0                  0.0              92000.0   
2            123000.0                  NaN             123000.0   
3             93000.0                  NaN              93000.0   
4             68500.0                  NaN              68500.0   

   assessed_value  common_level_ratio  fair_market_value  state_tax_amount  \
0             NaN                 NaN           30699.84             589.0   
1             NaN                 NaN           63936.00             920.0   
2             NaN                 NaN           43066.56            1230.0   
3             NaN                 NaN           45344.64             930.0   
4             NaN                 NaN          305913.60             685.0   

   state_tax_percent  local_tax_amount  local_tax_percent  \
0              100.0            1767.0              100.0   
1              100.0            2760.0              100.0   
2              100.0            3690.0              100.0   
3              100.0            2790.0              100.0   
4              100.0            2055.0              100.0   

   adjusted_cash_consideration  adjusted_other_consideration  \
0                      58900.0                           NaN   
1                      92000.0                           NaN   
2                     123000.0                           NaN   
3                      93000.0                           NaN   
4                      68500.0                           NaN   

   adjusted_total_consideration  adjusted_assessed_value  \
0                       58900.0                      NaN   
1                       92000.0                      NaN   
2                      123000.0                      NaN   
3                       93000.0                      NaN   
4                       68500.0                      NaN   

   adjusted_fair_market_value  adjusted_state_tax_amount  \
0                    30699.84                      589.0   
1                    63936.00                      920.0   
2                    43066.56                     1230.0   
3                    45344.64                      930.0   
4                   305913.60                      685.0   

   adjusted_local_tax_amount receipt_num         receipt_date  \
0                     1767.0         NaN  2004-06-07 00:00:00   
1                     2760.0         NaN  2000-06-07 00:00:00   
2                     3690.0         NaN  2004-06-08 00:00:00   
3                     2790.0         NaN  2004-06-08 00:00:00   
4                     2055.0         NaN  2004-06-07 00:00:00   

        recording_date        document_date condo_name unit_num  address_low  \
0  2004-06-07 00:00:00  2004-04-02 00:00:00        NaN      NaN       3083.0   
1  2000-06-07 00:00:00 

In [21]:
# Saving classified deeds to speed up future processing time 
deeds_for_save = deeds_multi
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].fillna(pd.Timedelta('999 days'))
deeds_for_save['elapsed'] = deeds_for_save['elapsed'].dt.days.astype('int16')
deeds_for_save['display_date_y'] = deeds_for_save['display_date_y'].astype(str)


In [22]:
# # saving final list of grantees
# with open("data/rtt_investors_classified.geojson",
#           'w') as f:
#     f.write(deeds_for_save.to_json())

FileNotFoundError: [Errno 2] No such file or directory: 'data/rtt_investors_classified.geojson'

## 5. Reimport geojson of classified deeds

To reimport classified deeds use gpd.read_file and then set the coordinate reference system:

In [3]:
# read in classified deeds file
rtt_classified = gpd.read_file("../00_data/PROCESSED/realestate_tax_transfers/rtt_investors_classified_2021.geojson")

KeyboardInterrupt: 

In [143]:
# Specify coordinate reference system
rtt_classified.crs = 'epsg:2272'